In [1]:
from esm_viz.visualization import general

In [2]:
from esm_viz.esm_viz import read_simulation_config

In [4]:
config = read_simulation_config("/home/csys/pgierz/.config/monitoring/LGM-011.yaml")

In [5]:
log = general.get_log_output(config)

In [7]:
log_df = general.generate_dataframe_from_esm_logfile(log)

In [11]:
diffs = general.compute_throughput(log_df)[2]

/home/csys/pgierz/esm-viz/esm_viz/visualization/general.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  starts.drop_duplicates(subset="Run Number", keep="last", inplace=True)


In [12]:
import datetime

In [13]:
throughput = datetime.timedelta(1) / diffs["Wall Time"].mean()

In [14]:
throughput

17.241045090122583

In [17]:
exp = config["basedir"]
model = config["model"].lower()
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(config["host"], username=config["user"])
date_filename = exp.split("/")[-1] + "_" + model + ".date"

In [16]:
import paramiko

In [18]:
date_filename

'LGM_011_awicm.date'

In [19]:
remote_command = ("cd " + config["basedir"] + "/scripts/; cat " + date_filename + " |awk '{ print $2 }'")
remote_command

"cd /work/ba0989/a270077/AWICM_PISM/LGM_011/scripts/; cat LGM_011_awicm.date |awk '{ print $1 }'"

In [57]:
stdin, stdout, stderr = client.exec_command(remote_command)

In [21]:
current_date = int(stdout.readlines()[0][:-5])

In [60]:
remote_command = ("cd " + config["basedir"] + "/scripts/; cat " + date_filename + " |awk '{ print $2 }'")
stdin, stdout, stderr = client.exec_command(remote_command)
current_run = int(stdout.readlines()[0])

In [61]:
current_run

176

In [23]:
runscript_file = config.get("runscript", config["basedir"] + "/scripts/*run")

In [24]:
start_year = client.exec_command("grep INITIAL_DATE_" + model + " " + runscript_file)[1].readlines()[0]

In [29]:
final_year = client.exec_command("grep FINAL_DATE_" + model + " " + runscript_file)[1].readlines()[0]

In [30]:
run_size = client.exec_command("grep NYEAR_" + model + " " + runscript_file)[1].readlines()[0]

In [31]:
start_year = int(start_year.split("=")[1].split("-")[0])

In [32]:
final_year = int(final_year.split("=")[1].split("-")[0])

In [33]:
run_size = int(general.stripComments(" ".join(run_size.split("=")[1].split())))

In [37]:
total_number_of_runs = int((final_year - start_year) / run_size)

In [38]:
total_number_of_runs

2000

In [39]:
years_per_day = throughput

In [41]:
years_left = final_year - current_date

In [42]:
years_left

1825

In [43]:
days_left = years_left / years_per_day

In [44]:
days_left

105.85205191798639

In [46]:
finishing_date = datetime.datetime.now() + datetime.timedelta(days=days_left)

In [47]:
finishing_date

datetime.datetime(2019, 12, 4, 9, 41, 22, 662152)

In [64]:
r_bar = (" "+ str(current_run)+ "/" + str(total_number_of_runs)+ ", Throughput ~"+ str(np.round(years_per_day, 2))+ "runs/day")

In [50]:
import numpy as np

In [65]:
r_bar

' 176/2000, Throughput ~17.24runs/day'

In [54]:
current_date

2025